<h1>Instalamos y Importamos las librerias<h1>

In [ ]:
#Instalar gecko
#from webdriver_manager.firefox import GeckoDriverManager
#GeckoDriverManager().install()
#!pip install requests lxml

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import os
import json
import requests
import base64


<h1>Configuracion de las rutas<h1>

In [ ]:
# Configuración de rutas
GECKO_DRIVER_PATH = 'C:/Users/constantin.madalin.i/mis-binarios/geckodriver.exe'
FIREFOX_BINARY_PATH = 'C:/Program Files/Mozilla Firefox/firefox.exe'
SCREENSHOT_DIR = 'buscados'
JSON_FILE = 'data.json'
BASE_URL = "https://eumostwanted.eu"

<h1>Configuracion del Firefox<h1>

In [ ]:
# Configurar Firefox
options = Options()
options.binary_location = FIREFOX_BINARY_PATH
service = Service(executable_path=GECKO_DRIVER_PATH)
driver = webdriver.Firefox(service=service, options=options)

<h1>Definimos las variables<h1>

In [ ]:
def wait_until(by, path, timeout=10):
    WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, path)))

def get_element_text(by, value):
    try:
        return driver.find_element(by, value).text
    except NoSuchElementException:
        return None

def get_element_attribute(by, value, attribute):
    try:
        return driver.find_element(by, value).get_attribute(attribute)
    except NoSuchElementException:
        return None

def get_image_base64(by, value, attribute):
    try:
        image_url = get_element_attribute(by, value, attribute)
        if image_url:
            response = requests.get(image_url)
            return base64.b64encode(response.content).decode('utf-8')
    except Exception:
        return None

def make_screenshot(index, name):
    if not os.path.exists(SCREENSHOT_DIR):
        os.makedirs(SCREENSHOT_DIR)
    driver.save_screenshot(os.path.join(SCREENSHOT_DIR, f"{index}_{name}.png"))

<h1>Definicion de los Xpaths<h1>

In [ ]:
# Definición de XPaths
class WantedFields:
    fields = {
        "wantedItem": "[class^='wantedItem']",
        "name": "//div[contains(@class, 'field--name-field-w-first-name')]/div[@class='field__item']",
        "crime": "//div[contains(@class, 'field--name-field-crime')]/div[@class='field__items']/div[@class='field__item']",
        "gender": "//div[contains(@class, 'field--name-field-gender')]/div[@class='field__item']",
        "height": "//div[contains(@class, 'field--name-field-approximate-height')]/div[@class='field__item']",
        "eyecolor": "//div[contains(@class, 'field--name-field-eye-colour')]/div[@class='field__item']",
        "birthdate": "//div[contains(@class, 'field--name-field-date-of-birth')]/div[@class='field__item']",
        "nationality": "//div[contains(@class, 'field--name-field-nationality')]/div[@class='field__items']/div[@class='field__item']",
        "picture": "//div[contains(@class, 'field--name-field-picture')]//img[1]",
    }

<h1>Entrar a la pagina y cerrar las cookies<h1>

In [ ]:
# Navegación al sitio web
driver.get(BASE_URL)
wait_until(By.ID, 'sort_close')
driver.find_element(By.ID, 'sort_close').click()

<h1>Cogiendo los datos de la persona y hacer una captura a la pantalla<h1>

In [ ]:
wanted = WantedFields()
wanted_list = driver.find_element(By.CLASS_NAME, 'wanted_list')
wanted_items = wanted_list.find_elements(By.CSS_SELECTOR, wanted.fields["wantedItem"])
params = [item.get_dom_attribute('href') for item in wanted_items if item.get_dom_attribute('href')]

datafinal = {"buscados": []}

for index, url_suspect in enumerate(params):
    full_url = url_suspect if url_suspect.startswith("http") else BASE_URL + url_suspect
    driver.get(full_url)
    wait_until(By.XPATH, wanted.fields["picture"])

    data = {
        "name": get_element_text(By.XPATH, wanted.fields["name"]),
        "crime": get_element_text(By.XPATH, wanted.fields["crime"]),
        "gender": get_element_text(By.XPATH, wanted.fields["gender"]),
        "height": get_element_text(By.XPATH, wanted.fields["height"]),
        "eyecolor": get_element_text(By.XPATH, wanted.fields["eyecolor"]),
        "birthdate": get_element_text(By.XPATH, wanted.fields["birthdate"]),
        "nationality": get_element_text(By.XPATH, wanted.fields["nationality"]),
        "picture": get_image_base64(By.XPATH, wanted.fields["picture"], 'src')
    }
    datafinal["buscados"].append(data)
    make_screenshot(index, data["name"] if data["name"] else "unknown")

<h1>Guardamos en un Json la información<h1>

In [ ]:
# Guardar en JSON
with open(JSON_FILE, 'w', encoding='utf-8') as outfile:
    json.dump(datafinal, outfile, ensure_ascii=False, indent=4)

driver.quit()